In [ ]:
! pip install google-cloud-translate pandas requests bs4 google-generativeai

In [6]:
# Lưu vào file csv
import requests
import time
import os
from bs4 import BeautifulSoup
from urllib.parse import urlencode
import google.generativeai as genai
from openai import OpenAI
import pandas as pd

## Điền thông tin thuốc còn thiếu bằng cách tìm kiếm trên gg search

In [ ]:
# ggsearchAPI = os.environ.get('GGSEARCH_API_KEY')
# ggsearchCSE = os.environ.get('GGSEARCH_CSE_ID')

ggsearchAPI = os.getenv('GGSEARCH_API_KEY')
ggsearchCSE = os.getenv('GGSEARCH_CSE_ID')
api_key = ggsearchAPI   # Thay thế bằng API key của bạn
cse_id = ggsearchCSE    # Thay thế bằng Custom Search Engine ID của bạn

base_path = os.path.join(os.getcwd(), 'Medicines-Data-Imputation/PreData/RAW_vn_medicines_202405171011.csv')
global missing_imputation
missing_imputation_and_standard = os.path.join(os.getcwd(), 'Medicines-Data-Imputation/DataOut/missing_activeIngredient_imputation_and_standard_unit_vn.csv')

# Hàm tìm kiếm Google và lấy thông tin dạng bào chế
def search_google(query, api_key, cse_id):
    query_params = {
        'q': f"{query} site:thuocbietduoc.com.vn",
        'key': api_key,
        'cx': cse_id
    }
    url = f"https://www.googleapis.com/customsearch/v1?{urlencode(query_params)}"
  
    retries = 5
    for i in range(retries):
        try:
            response = requests.get(url)
            response.raise_for_status()  # Ném lỗi HTTPError cho các phản hồi xấu

            if response.status_code == 200:
                results = response.json()
                
                if 'items' in results and results['items']:
                    first_result = results['items'][0]
                    link = first_result['link']
                    
                    response_detail = requests.get(link)
                    response_detail.raise_for_status()
                    
                    if response_detail.status_code == 200:
                        soup_detail = BeautifulSoup(response_detail.text, 'html.parser')
                        all_spans = soup_detail.find_all('span')
                        for span in all_spans:
                            if 'Dạng bào chế:' in span.get_text():
                                dosage_form = span.find_next('span', class_='textdetaildrg')
                                if dosage_form:
                                    return dosage_form.get_text()
                        return None
                else:
                    return None
            elif response.status_code == 429:
                wait_time = 2 ** i
                print(f"Đã bị giới hạn. Chờ {wait_time} giây.")
                time.sleep(wait_time)
            else:
                return None
        
        except requests.exceptions.RequestException as e:
            print(f"Lỗi yêu cầu: {e}")
            if i < retries - 1:  # Nếu không phải lần thử cuối cùng, chờ và thử lại
                wait_time = 2 ** i
                print(f"Chờ {wait_time} giây trước khi thử lại.")
                time.sleep(wait_time)
            else:
                return None

# Đọc dữ liệu từ tệp CSV
df = pd.read_csv(base_path)

# Duyệt qua các hàng của DataFrame và tìm kiếm thông tin dạng bào chế nếu ô trống
for index, row in df.iterrows():
    if pd.isna(row['form']):
        medicine_name = row['name']  # Giả sử cột tên thuốc là 'name'
        dosage_form = search_google(medicine_name, api_key, cse_id)
        if dosage_form:
            df.at[index, 'form'] = dosage_form

# Lưu lại DataFrame vào tệp CSV mới
output_file_path = missing_imputation_and_standard
df.to_csv(output_file_path, index=False)

## Chuẩn hóa đơn vị của nồng độ/ định lượng

In [2]:
# Sử dung ollama
! curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
######################################################################## 100.0%#=#=#                                                                         
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to render group...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> Enabling and starting ollama service...
Created symlink /etc/systemd/system/default.target.wants/ollama.service → /etc/systemd/system/ollama.service.
>>> NVIDIA GPU installed.


### Model library

Ollama supports a list of models available on [ollama.com/library](https://ollama.com/library 'ollama model library')

Here are some example models that can be downloaded:

| Model              | Parameters | Size  | Download                       |
| ------------------ | ---------- | ----- | ------------------------------ |
| Llama 3            | 8B         | 4.7GB | `ollama run llama3`            |
| Llama 3            | 70B        | 40GB  | `ollama run llama3:70b`        |
| Phi 3 Mini         | 3.8B       | 2.3GB | `ollama run phi3`              |
| Phi 3 Medium       | 14B        | 7.9GB | `ollama run phi3:medium`       |
| Gemma 2            | 9B         | 5.5GB | `ollama run gemma2`            |
| Gemma 2            | 27B        | 16GB  | `ollama run gemma2:27b`        |
| Mistral            | 7B         | 4.1GB | `ollama run mistral`           |
| Moondream 2        | 1.4B       | 829MB | `ollama run moondream`         |
| Neural Chat        | 7B         | 4.1GB | `ollama run neural-chat`       |
| Starling           | 7B         | 4.1GB | `ollama run starling-lm`       |
| Code Llama         | 7B         | 3.8GB | `ollama run codellama`         |
| Llama 2 Uncensored | 7B         | 3.8GB | `ollama run llama2-uncensored` |
| LLaVA              | 7B         | 4.5GB | `ollama run llava`             |
| Solar              | 10.7B      | 6.1GB | `ollama run solar`             |

> Note: You should have at least 8 GB of RAM available to run the 7B models, 16 GB to run the 13B models, and 32 GB to run the 33B models.

In [ ]:
# Gọi model gemmma 2
! ollama run gemma2

In [ ]:
! pip install openai

In [ ]:
returnaw_fo = "RAW_foreign_warehouses_202405171033.csv"
raw_vn = "RAW_vn_medicines_202405171011.csv"
prefix = os.path.join(os.getcwd(), f'Medicines-Data-Imputation/PreData/{raw_fo}')
df = pd.read_csv(prefix)
for index, row in df.iterrows():
    print(row['strength'][1:-1])

In [8]:
raw_fo = "RAW_foreign_warehouses_202405171033.csv"
raw_vn = "RAW_vn_medicines_202405171011.csv"

prefix = os.path.join(os.getcwd(), f'Medicines-Data-Imputation/PreData/{raw_fo}')

conten_vn = """Bạn là một trợ lý AI có khả năng chuẩn hóa nồng độ, định lượng.
                  Ví dụ khi đầu vào là 1g(khối lượng) thì trả về "1000mg".
                  khi đầu vào là 0,02l(thể tích) thì trả về "20ml".
                  Khi đầu vào là 1g/2ml(khối lượng/thể tích) thì trả về "500mg/ml".
                  Đầu vào có thể có nhiều giá trị như "1,0 g, 3,0 g, 3,0 g, 3,0 g " thì bạn cũng chuyển nó về đúng đơn vị mg "1000mg, 3000mg, 3000mg, 3000mg".
                  các đơn vị khác như %, iu, ui hoặc "không có đơn vị" sẽ giữ nguyên.
                  Nếu đầu vào là kí tự kì lạ hoặc không thể chuẩn hóa thì trả về khoảng trống "". """

content_fo = """Bạn là một trợ lý AI có khả năng chuẩn hóa nồng độ, định lượng.
                  Ví dụ khi đầu vào là ["1g"](khối lượng) thì trả về "1000mg".
                  khi đầu vào là ["0,02l"](thể tích) thì trả về "20ml".
                  Khi đầu vào là ["1g/2ml"](khối lượng/thể tích) thì trả về "500mg/ml".
                  Đầu vào có thể có nhiều giá trị như ["1,0 g", "3,0 g", "3,0 g", "3,0 g "] thì bạn cũng chuyển nó về đúng đơn vị mg "1000mg, 3000mg, 3000mg, 3000mg".
                  các đơn vị khác như %, iu, ui hoặc "không có đơn vị" sẽ giữ nguyên.
                  Nếu đầu vào là kí tự kì lạ hoặc không thể chuẩn hóa thì trả về khoảng trống "". """

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

def standardize_unit(strength):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": content_fo
            },
            {
                'role': 'user',
                'content': f"""Chuẩn hóa nồng độ, định lượng "{strength}". Lưu ý: Bạn chỉ cần trả về đúng kết quả chuẩn hóa, ghi trên 1 dòng"""
            }
        ],
        temperature = 0.7,
        max_tokens = 256,
        
        #request_timeout=config.REQUEST_TIMEOUT,
        model='gemma2',
    )
    response = chat_completion.choices[0].message.content
    return response

def is_standard_unit(value):
  if not value or value == "nan" or pd.isnull(value):
    return True
  units = ["mg", "ml", "mg/ml", "%", "iu", "ui"]
  for unit in units:
    value = str(value).lower()
    if unit in value:
      return True
  return False

# Đọc file CSV
df = pd.read_csv(prefix)

# Chuẩn hóa đơn vị trong cột "strength"
for index, row in df.iterrows():
    if prefix == os.path.join(os.getcwd(), f'Medicines-Data-Imputation/PreData/{raw_fo}'):
        strength = row['strength'][1:-1]
    else: 
        strength = row['strength']
    if not is_standard_unit(strength):
        # time.sleep(0.1)
        standardized_strength = standardize_unit(strength)
        if standardized_strength:
            df.at[index, 'strength'] = standardized_strength
            print(f"Đã chuẩn hóa: {strength} -> {standardized_strength}")

# df.to_csv(missing_imputation_and_standard, index=False)
# df

Đã chuẩn hóa: "50mcg", "500mcg" -> Okay, I'm ready! Give me your input and I'll standardize the concentration and quantities for you.  

Please remember: I will convert mass to milligrams (mg), volume to milliliters (ml), and perform those conversions on individual terms as well as entire lists. Other units like %, iu, ui or "no unit" will be left unchanged. 

Đã chuẩn hóa: "1g" -> 1000mg 

Đã chuẩn hóa: "400mcg" -> 400mg  

Đã chuẩn hóa: "250mcg", "50mcg", "500mcg" -> Ok! Dựa vào hướng dẫn của bạn, tôi sẽ xử lí các đầu vào và trả về kết quả chuẩn hóa. Đừng ngần ngại đưa ra những ví dụ để tôi có thể học hỏi và cải thiện hiệu suất của mình.   



In [9]:
vn = "vn"
fo = "foreign"
missing_imputation_and_standard = os.path.join(os.getcwd(), f'Medicines-Data-Imputation/DataOut/missing_activeIngredient_imputation_and_standard_unit_{vn}.csv')
if not os.path.exists(os.path.join(os.getcwd(), 'Medicines-Data-Imputation/DataOut')):
    os.makedirs(os.path.join(os.getcwd(), 'Medicines-Data-Imputation/DataOut'))
df.to_csv(missing_imputation_and_standard, index=False)

In [7]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

content_in = """Bạn là một trợ lý AI có khả năng phân tích thông tin thuốc và chuẩn hóa nồng độ, định lượng. Nhiệm vụ của bạn là đọc một chuỗi văn bản đầu vào mô tả tên một loại thuốc hoặc hoạt chất và định lượng của nó, sau đó tách biệt thông tin này thành hai phần: công thức hóa học và phần định lượng, được phân cách bởi dấu phẩy.
                Ví dụ:
                * Đầu vào: ". acetylcysteine 100 mg sachet" 
                * Đầu ra: "C5H9NO3S, 100mg"
                Hãy tuân theo các quy tắc sau:
                * **Công thức hóa học:** Ưu tiên chuyển đổi tên thuốc hoặc hoạt chất thành công thức hóa học nếu tìm thấy tương ứng, chỉ ghi công thức hóa học trên 1 dòng như ví dụ "C5H9NO3S". Nếu không tìm thấy công thức hóa học, hãy giữ nguyên tên ban đầu.
                * **Chuẩn hóa định lượng:**
                    * Chuyển đổi khối lượng sang "mg": Ví dụ: "1g" thành "1000mg", "0.5g" thành "500mg".
                    * Chuyển đổi thể tích sang "ml": Ví dụ: "0.02l" thành "20ml".
                    * Chuyển đổi nồng độ sang "mg/ml": Ví dụ: "1g/2ml" thành "500mg/ml".
                    * Các đơn vị khác như %, iu, ui hoặc không có đơn vị, hãy giữ nguyên.
                * **Xử lý nhiều giá trị:** Nếu đầu vào có nhiều giá trị, hãy chuyển đổi tất cả chúng sang cùng một đơn vị và phân cách bằng dấu phẩy. Ví dụ: "1,0 g, 3,0 g" thành "1000mg, 3000mg".
                * **Xử lý lỗi:** Nếu đầu vào là kí tự kì lạ hoặc không thể chuẩn hóa, hãy trả về khoảng trống "".
                Bây giờ hãy áp dụng các quy tắc trên để xử lý đầu vào sau:"""

def active_ingredient(ingredient):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": content_in
            },
            {
                'role': 'user',
                'content': ingredient
            }
        ],
        temperature = 0.7,
        max_tokens = 256,
        
        #request_timeout=config.REQUEST_TIMEOUT,
        model='gemma2',
    )
    response = chat_completion.choices[0].message.content
    return response
vn = "vn"
fo = "foreign"
missing_imputation_and_standard = os.path.join(os.getcwd(), f'Medicines-Data-Imputation/DataOut/missing_activeIngredient_imputation_and_standard_unit_{vn}.csv')
df = pd.read_csv(missing_imputation_and_standard)

for index, row in df.iterrows():
    ingredient = row['activeIngredient']
    if not pd.isna(ingredient):
        standardized_ingredient = active_ingredient(ingredient)
        if standardized_ingredient:
            df.at[index, 'activeIngredient'] = standardized_ingredient
            print(f"Đã chuẩn hóa: {ingredient} -> {standardized_ingredient}")

Đã chuẩn hóa: Mỗi 5ml chứa: Fexofenadin hydrochlorid 300mg -> C17H24ClN2O3.HCl , 300mg/5ml 

Đã chuẩn hóa: Triamcinolon 4mg -> C22H29O5, 4mg  

Đã chuẩn hóa: Oxacillin (dưới dạng Oxacillin Natri) 500mg -> C18H26N4O9SNa, 500mg  

Đã chuẩn hóa: Mỗi 60ml dung dịch chứa Piracetam 6000mg -> C7H13NO2, 6000mg/60ml  

Đã chuẩn hóa: Mỗi 60ml dung dịch chứa Piracetam 6000mg -> C<sub>7</sub>H<sub>13</sub>NO<sub>2</sub>S, 6000mg/60ml 

Đã chuẩn hóa: Mỗi viên chứa bột mịn cao hỗn hợp dược liệu 0,33 g (tương đương: Tế tân 0,48 g; Sinh địa 0,33 g; Đương quy 0,27 g; Thăng ma 0,26 g; Mẫu đơn bì 0,2 g; Hoàng liên 0,176 g; Bột mịn Thăng ma 0,07 g; Bột mịn Đương quy 0,06 g; Bột mịn Hoàng liên 0,024 g); Bột mịn Tế tân 0,02 g ->  "",  0.33g,0.48g,0.33g,0.27g,0.26g,0.2g,0.176g,0.07g,0.06g,0.024g,0.02g  

Đã chuẩn hóa: Mỗi 5ml chứa: Desloratadin 2,5 mg -> C<sub>22</sub>H<sub>23</sub>ClN<sub>2</sub>O<sub>4</sub>, 2.5mg/5ml  

Đã chuẩn hóa: Mỗi 5ml dung dịch chứa: Sắt nguyên tố (dưới dạng Sắt (III) hydroxyd pol

In [8]:
vn = "vn"
fo = "foreign"
missing_imputation_and_standard = os.path.join(os.getcwd(), f'Medicines-Data-Imputation/DataOut/convert_ingredient_{vn}.csv')
if not os.path.exists(os.path.join(os.getcwd(), 'Medicines-Data-Imputation/DataOut')):
    os.makedirs(os.path.join(os.getcwd(), 'Medicines-Data-Imputation/DataOut'))
df.to_csv(missing_imputation_and_standard, index=False)